In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split


In [ ]:
!nvidia-smi


Thu Dec  4 18:42:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import kagglehub

path = kagglehub.dataset_download("grassknoted/asl-alphabet")
print("Dataset downloaded to:", path)

data_dir = "/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"

Using Colab cache for faster access to the 'asl-alphabet' dataset.
Dataset downloaded to: /kaggle/input/asl-alphabet


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load dataset
full_train = datasets.ImageFolder(data_dir, transform=transform)

# Labels and indices for split
labels = np.array(full_train.targets)
indices = np.arange(len(full_train))

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
data_dir = "/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"


In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [ ]:
train_idx, val_idx = train_test_split(
    indices,
    test_size=0.2,
    stratify=labels,
    random_state=429
)

train_dataset = Subset(full_train, train_idx)
val_dataset = Subset(full_train, val_idx)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)


In [ ]:
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Freeze all params
for param in model.parameters():
    param.requires_grad = False

# Freeze BN layers: weights + running stats
for module in model.modules():
    if isinstance(module, nn.BatchNorm2d):
        module.eval()
        module.track_running_stats = False  # required to stop stat updates
        for param in module.parameters():
            param.requires_grad = False

# Replace classifier head
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(full_train.classes))
model.fc.requires_grad = True

model = model.to(device)


In [ ]:
import tqdm

EPOCHS = 3

# Define optimizer and criterion
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    # Wrap train_loader with tqdm for a progress bar
    for imgs, labels in tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} Training"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    train_loss /= total

    # ---- Validation ----
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        # Wrap val_loader with tqdm for a progress bar
        for imgs, labels in tqdm.tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} Validation"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * imgs.size(0)
            _, preds = outputs.max(1)
            val_correct += preds.eq(labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    val_loss /= val_total

    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

Epoch 1/3 Validation: 100%|██████████| 272/272 [00:46<00:00,  5.79it/s]


Epoch 1/3 | Train Loss: 0.8427, Train Acc: 0.8311 | Val Loss: 2.4699, Val Acc: 0.3061


Epoch 2/3 Validation: 100%|██████████| 272/272 [00:45<00:00,  5.93it/s]


Epoch 2/3 | Train Loss: 0.2748, Train Acc: 0.9431 | Val Loss: 2.6465, Val Acc: 0.3369


Epoch 3/3 Validation: 100%|██████████| 272/272 [00:47<00:00,  5.75it/s]

Epoch 3/3 | Train Loss: 0.1871, Train Acc: 0.9583 | Val Loss: 2.7685, Val Acc: 0.3327


In [ ]:
from sklearn.metrics import f1_score

def compute_f1(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = outputs.max(1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return f1_score(all_labels, all_preds, average='macro')


In [ ]:
final_f1 = compute_f1(model, val_loader, device)
print("Final T-A Macro F1:", final_f1)

Final T-A Macro F1: 0.3350175882642035
